In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import Pipeline
from dotenv import load_dotenv
import os
from transformers import AutoTokenizer, AutoModel


load_dotenv()
DATASET_PATH = f'{os.getenv("DATASETS_PATH")}depression_ds.csv'
df = pd.read_csv(DATASET_PATH)
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=13)

/home/ksaff/miniconda3/envs/smlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from bert_transformer import BertTransformer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained("bert-base-uncased").to('cuda:0')

bert_transformer = BertTransformer(tokenizer, model, 500)

In [3]:
classifier = svm.SVC(kernel='linear')

model = Pipeline([
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
        ])

In [4]:
train_X = df_train['clean_text']
train_y = df_train['is_depression']

model = Pipeline([
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
        ])
model.fit(train_X, train_y)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/home/ksaff/Desktop/Small_ML_Projects/binary_depression/bert_transformer.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(tokenized_text).unsqueeze(0).to(self.device),


Pipeline(steps=[('vectorizer',
                 BertTransformer(bert_model=BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder...
                                 bert_tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),
                                 max_length=500)),
                ('classifier', SVC(kernel='linear'))])

In [5]:
test_X = df_test['clean_text']
test_y = df_test['is_depression']

print(len(test_X))

predictions = model.predict(test_X)

1547


/home/ksaff/Desktop/Small_ML_Projects/binary_depression/bert_transformer.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(tokenized_text).unsqueeze(0).to(self.device),


In [11]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(predictions, test_y)

print(f"Succes rate: {round(acc * 100, 2)} %")

Succes rate: 93.99 %
